<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Sprint Challenge
## *Data Science Unit 4 Sprint 1*

After a week of Natural Language Processing, you've learned some cool new stuff: how to process text, how turn text into vectors, and how to model topics from documents. Apply your newly acquired skills to one of the most famous NLP datasets out there: [Yelp](https://www.yelp.com/dataset/challenge). As part of the job selection process, some of my friends have been asked to create analysis of this dataset, so I want to empower you to have a head start.  

The real dataset is massive (almost 8 gigs uncompressed). I've sampled the data for you to something more managable for the Sprint Challenge. You can analyze the full dataset as a stretch goal or after the sprint challenge. As you work on the challenge, I suggest adding notes about your findings and things you want to analyze in the future.

## Challenge Objectives
*Successfully complete these all these objectives to earn a 2. There are more details on each objective further down in the notebook.*
* <a href="#p1">Part 1</a>: Write a function to tokenize the yelp reviews
* <a href="#p2">Part 2</a>: Create a vector representation of those tokens
* <a href="#p3">Part 3</a>: Use your tokens in a classification model on yelp rating
* <a href="#p4">Part 4</a>: Estimate & Interpret a topic model of the Yelp reviews

In [49]:
import pandas as pd
import spacy
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import SGDClassifier, LogisticRegression, RidgeClassifier
from sklearn.pipeline import Pipeline
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import GridSearchCV

In [2]:
yelp = pd.read_json('./data/review_sample.json', lines=True)
yelp.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,nDuEqIyRc8YKS1q1fX0CZg,1,2015-03-31 16:50:30,0,eZs2tpEJtXPwawvHnHZIgQ,1,"BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...",10,n1LM36qNg4rqGXIcvVXv8w
1,eMYeEapscbKNqUDCx705hg,0,2015-12-16 05:31:03,0,DoQDWJsNbU0KL1O29l_Xug,4,Came here for lunch Togo. Service was quick. S...,0,5CgjjDAic2-FAvCtiHpytA
2,6Q7-wkCPc1KF75jZLOTcMw,1,2010-06-20 19:14:48,1,DDOdGU7zh56yQHmUnL1idQ,3,I've been to Vegas dozens of times and had nev...,2,BdV-cf3LScmb8kZ7iiBcMA
3,k3zrItO4l9hwfLRwHBDc9w,3,2010-07-13 00:33:45,4,LfTMUWnfGFMOfOIyJcwLVA,1,We went here on a night where they closed off ...,5,cZZnBqh4gAEy4CdNvJailQ
4,6hpfRwGlOzbNv7k5eP9rsQ,1,2018-06-30 02:30:01,0,zJSUdI7bJ8PNJAg4lnl_Gg,4,"3.5 to 4 stars\n\nNot bad for the price, $12.9...",5,n9QO4ClYAS7h9fpQwa5bhA


## Part 1: Tokenize Function
<a id="#p1"></a>

Complete the function `tokenize`. Your function should
- accept one document at a time
- return a list of tokens

You are free to use any method you have learned this week.

In [3]:
nlp = spacy.load('en_core_web_lg')

In [4]:
def tokenize(doc):
    
    return [token.text for token in nlp(doc) if not token.is_stop and not token.is_punct and not token.pos == 'PRON']

## Part 2: Vector Representation
<a id="#p2"></a>
1. Create a vector representation of the reviews
2. Write a fake review and query for the 10 most similiar reviews, print the text of the reviews. Do you notice any patterns?
    - Given the size of the dataset, it will probably be best to use a `NearestNeighbors` model for this. 

In [5]:
yelp.text.head()

0    BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...
1    Came here for lunch Togo. Service was quick. S...
2    I've been to Vegas dozens of times and had nev...
3    We went here on a night where they closed off ...
4    3.5 to 4 stars\n\nNot bad for the price, $12.9...
Name: text, dtype: object

In [6]:
data = yelp['text'].apply(tokenize)

In [7]:
data

0       [BEWARE, FAKE, FAKE, FAKE, small, business, Lo...
1       [Came, lunch, Togo, Service, quick, Staff, fri...
2       [Vegas, dozens, times, stepped, foot, Circus, ...
3       [went, night, closed, street, party, best, act...
4       [3.5, 4, stars, \n\n, bad, price, $, 12.99, lu...
5       [Tasty, fast, casual, Latin, street, food,  , ...
6       [absolutely, amazing, incredible, production, ...
7       [Came, Pho, enjoyed,  , got, 9:00pm, busy, got...
8       [Absolutely, Unique, experience, nail, shop, f...
9       [Wow, walked, sat, bar, 10, minutes, bartender...
10      [popped, dinner, yesterday, reservation, despi...
11      [Thw, worst, stay, ended, paying, 700, 800, 2,...
12      [Great, friendly, customer, service, quality, ...
13      [food, great,  , super, busy, server, attentiv...
14      [Talk, getting, ripped, charged, $, 420, shut,...
15      [Girls, night, tonight, kid, decided, drive, h...
16      [Stopped, drinks, flying, Charlotte, weeks, ba...
17      [excel

In [8]:
vect = [nlp(doc).vector for doc in yelp['text']]

In [25]:
fake = "I did not like this restaurant. The service was ok but the food was cold and bland."


In [26]:
nn = NearestNeighbors(n_neighbors=10, algorithm='ball_tree')
nn.fit(vect)

NearestNeighbors(algorithm='ball_tree', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=None, n_neighbors=10, p=2, radius=1.0)

In [27]:
new = nlp(fake).vector

In [28]:
ten_nearest = nn.kneighbors([new])[1]

In [29]:
for review in ten_nearest:
    print(yelp['text'].iloc[review])

868     It was just ok.  I liked the setting and the f...
8051    The food tasted like the cook was pretending t...
9609    I wasn't impress. We sat along the canal and t...
753     The service was horrible. Everything took too ...
922     Came here twice as it was right beside the hot...
2199    Came for lunch and didn't see much of a crowd ...
1296    Came for dinner with some friends. Let's start...
5356    Definitely not the best Italian restaurant I'v...
2113    This location is actually very clean compared ...
5334    I had such High expectation for Toqué! and I w...
Name: text, dtype: object


## Part 3: Classification
<a id="#p3"></a>
Your goal in this section will be to predict `stars` from the review dataset. 

1. Create a piepline object with a sklearn `CountVectorizer` or `TfidfVector` and any sklearn classifier. Use that pipeline to estimate a model to predict `stars`. Use the Pipeline to predict a star rating for your fake review from Part 2. 
2. Tune the entire pipeline with a GridSearch

In [30]:
y = yelp.stars

In [44]:
vect = CountVectorizer(ngram_range=(1, 3),
                       max_features=75,
#                        tokenizer=tokenize,
                      )
sgd = SGDClassifier()
pipe = Pipeline([('vect', vect), ('mdl', sgd)])

In [45]:
pipe.fit(yelp['text'], y)

/Users/lambda_school_loaner_153/anaconda3/envs/unit4/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=75, min_df=1,
        ngram_range=(1, 3), preprocessor=None, stop_words=None,
        strip_a...m_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False))])

In [46]:
pipe.predict(tokenize(fake))

array([5, 5, 5, 5, 5, 5, 5])

Well that's shitty lol.

In [47]:
params = {'vect__ngram_range': [(1, 2), (1, 3)],
          'vect__max_features': [75, 100]}

In [52]:
cv = GridSearchCV(pipe, params)

In [53]:
cv.fit(yelp['text'], y)

/Users/lambda_school_loaner_153/anaconda3/envs/unit4/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/Users/lambda_school_loaner_153/anaconda3/envs/unit4/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=75, min_df=1,
        ngram_range=(1, 3), preprocessor=None, stop_words=None,
        strip_a...m_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'vect__ngram_range': [(1, 2), (1, 3)], 'vect__max_features': [75, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [55]:
cv.best_params_

{'vect__max_features': 75, 'vect__ngram_range': (1, 2)}

## Part 4: Topic Modeling

Let's find out what those yelp reviews are saying! :D

1. Estimate a LDA topic model of the review text
2. Create 1-2 visualizations of the results
    - You can use the most important 3 words of a topic in relevant visualizations. Refer to yesterday's notebook to extract. 
3. In markdown, write 1-2 paragraphs of analysis on the results of your topic model

__*Note*__: You can pass the DataFrame column of text reviews to gensim. You do not have to use a generator.

In [62]:
from gensim import corpora
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel
from gensim.parsing.preprocessing import STOPWORDS
from gensim.utils import simple_preprocess
import warnings
warnings.filterwarnings('ignore')


In [56]:
def tokenize(text):
    return [token for token in simple_preprocess(text) if token not in STOPWORDS]


In [73]:
tokens = [tokenize(t) for t in yelp['text']]

In [74]:
id2word = corpora.Dictionary(tokens)

In [75]:
id2word.filter_extremes(no_below=10, no_above=0.75)

In [76]:
corpus = [id2word.doc2bow(token) for token in tokens]

In [77]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   random_state=42,
                   num_topics=15,
                   passes=10,
                   workers=4,
                   )

In [79]:
import re
words = [re.findall(r'"([^"]*)"', topic[1]) for topic in lda.print_topics()]

In [80]:
topics = [' '.join(topic[0:5]) for topic in words]

In [82]:
topics

['great service place time love',
 'place ice food good eggs',
 'food good chicken like place',
 'said told called like time',
 'hair great time ve new',
 'food good great service place',
 'place service time minutes went',
 'time service place great got',
 'great food service place time',
 'good fries place great burger',
 'time car day got work',
 'room hotel time pool like',
 'food order got ordered like',
 'place food good love great',
 'pizza good place great like']

In [81]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.158334  0.018947       1        1  18.024380
5      0.114496  0.017148       2        1   8.675267
12     0.079535  0.107840       3        1   8.284332
11    -0.063839  0.052678       4        1   7.264898
3     -0.164762  0.055619       5        1   7.110874
6     -0.057336  0.107152       6        1   6.742859
10    -0.138979  0.001024       7        1   6.710000
13     0.110556 -0.048138       8        1   6.591577
7     -0.100523  0.022471       9        1   6.066681
4     -0.153636 -0.123575      10        1   5.342715
8      0.012064 -0.037765      11        1   4.758078
14     0.023114 -0.062049      12        1   4.255068
1      0.100983  0.024622      13        1   3.959064
9      0.073819 -0.087062      14        1   3.175187
0      0.006175 -0.048912      15        1   3.039021, topic_info=     Category         Freq        Term        Total  loglift  logprob
348   Default   755.000000       pizza   755.000000  30.0000  30.0000
188   Default  4184.000000       great  4184.000000  29.0000  29.0000
133   Default  4960.000000        food  4960.000000  28.0000  28.0000
37    Default  4582.000000        good  4582.000000  27.0000  27.0000
94    Default  1031.000000        room  1031.000000  26.0000  26.0000
122   Default  4687.000000       place  4687.000000  25.0000  25.0000
189   Default   395.000000        hair   395.000000  24.0000  24.0000
1246  Default   591.000000       fries   591.000000  23.0000  23.0000
1039  Default  1448.000000     chicken  1448.000000  22.0000  22.0000
1667  Default   631.000000         car   631.000000  21.0000  21.0000
1509  Default   466.000000       hotel   466.000000  20.0000  20.0000
671   Default   466.000000         ice   466.000000  19.0000  19.0000
318   Default  1247.000000   delicious  1247.000000  18.0000  18.0000
42    Default  3599.000000     service  3599.000000  17.0000  17.0000
309   Default   833.000000      cheese   833.000000  16.0000  16.0000
1029  Default  1596.000000     ordered  1596.000000  15.0000  15.0000
667   Default   563.000000       cream   563.000000  14.0000  14.0000
342   Default  1521.000000       order  1521.000000  13.0000  13.0000
702   Default   719.000000       store   719.000000  12.0000  12.0000
638   Default  1589.000000        love  1589.000000  11.0000  11.0000
1736  Default   527.000000      burger   527.000000  10.0000  10.0000
176   Default  1344.000000     amazing  1344.000000   9.0000   9.0000
168   Default  1144.000000   recommend  1144.000000   8.0000   8.0000
2011  Default   330.000000       tacos   330.000000   7.0000   7.0000
465   Default  1008.000000        told  1008.000000   6.0000   6.0000
111   Default  1854.000000        best  1854.000000   5.0000   5.0000
31    Default  3455.000000        time  3455.000000   4.0000   4.0000
22    Default   957.000000     minutes   957.000000   3.0000   3.0000
27    Default  1251.000000        said  1251.000000   2.0000   2.0000
353   Default   855.000000       sauce   855.000000   1.0000   1.0000
...       ...          ...         ...          ...      ...      ...
638   Topic15    96.124283        love  1589.751343   0.6879  -4.9040
932   Topic15    23.479925     healthy    96.572395   2.0795  -6.3135
188   Topic15   147.120926       great  4184.239746   0.1458  -4.4784
101   Topic15    63.497620       times   858.936218   0.8889  -5.3186
42    Topic15   121.228966     service  3599.113037   0.1029  -4.6719
2621  Topic15    20.916124         vet    78.584641   2.1700  -6.4291
645   Topic15    44.436874   selection   459.342896   1.1579  -5.6756
122   Topic15   119.130402       place  4687.744141  -0.1789  -4.6894
31    Topic15    98.488426        time  3455.003418  -0.0640  -4.8797
176   Topic15    65.497971     amazing  1344.030762   0.4722  -5.2876
661   Topic15    45.635956       worth   632.209961   0.8651  -5.6489
15    Topic15    85.123535   

Yeah I'm not super confident on the whole analysis of what's going on here. I know the 'topics' are peaks on a probability density function... Not too sure on how to use any of this at all.

## Stretch Goals

Complete one of more of these to push your score towards a three: 
* Incorporate named entity recognition into your analysis
* Compare vectorization methods in the classification section
* Analyze more (or all) of the yelp dataset - this one is v. hard. 
* Use a generator object on the reviews file - this would help you with the analyzing the whole dataset.
* Incorporate any of the other yelp dataset entities in your analysis (business, users, etc.)